In [2]:
import pandas as pd
from lxml import etree
import os
from bs4 import BeautifulSoup
import requests
import xml.etree.ElementTree as ET
import validators
import datetime
import re

In [5]:
#utils

def add_zeros(int_):
    if len(str(int_))<2:
        return '00'+str(int_)
    if len(str(int_))<3:
        return '0'+str(int_)
    if len(str(int_))==3:
        return str(int_)

def get_text_from_html(url__):
    from bs4 import BeautifulSoup
    res = requests.get(url__)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    return str(soup.noscript)

In [78]:
#regex
def check_dialog(text_):
    pat="(?:</p><p>.*?\(.*?\): —.*?\: —)"
    match=re.findall(pat, text_)
    if match is None :
        return 'no_dialog'
    else:
        for m in match:
            
            return m

def extract_dialog(full_text):    
    if full_text is None:
        return None
    else:
        dialogs=[(m.start(0), m.end(0)) for m in re.finditer('</p><p></p><p>(.+?): —', full_text)]
        out=[]
        if len(dialogs)==1:
            out.append(full_text[dialogs[0][0]:])
        if len(dialogs)>1:
            for i in range(0, len(dialogs)-1):
                out.append(full_text[dialogs[i][0]:dialogs[i+1][0]])
            out.append(full_text[dialogs[len(dialogs)-1][0]:])

        return out
    
def extract_party_name(dialogs):
    if dialogs is None:
        return None
    else:
        for i in dialogs:
            positions=[(m.start(0), m.end(0)) for m in re.finditer('(.+?): —', i)]
            party=re.findall('\((.+?)\)',i)
            name_aux=i[positions[0][1]-50:positions[0][1]]
            name=re.sub(r'\b\w{1,2}\b', '', name_aux).replace('.','').replace ('()','').replace('  ','').replace('<','').replace('>','').replace('/','').replace(': —','')
            if len(party)>0:
                party=party[0]
            return [party, name]

In [3]:
current_directory="C:\\Users\caperei"
cycle='14'
session='1'
data_directory=f"\portuguese_open_data\data\cycle\{cycle}\session\{session}\\numbers_dates_pages.csv"
input_data=pd.read_csv(r"C:\Users\caperei\portuguese_open_data\data\cyle\14\session\1\numbers_dates_pages.csv")

In [4]:
input_data.head(3)

,number,date,pages
0,1,25/10/2019,13
1,2,30/10/2019,124
2,3,31/10/2019,26


In [6]:
df = pd.DataFrame(columns=['legislature','cycle','session','number','date','page', 'text_1','url'])        
legislature='dar/01'
cycle='14'
session=['01'] #['01','02','03']
numbers_dates=list(zip(input_data.number, input_data.date,input_data.pages ))[0:1]
pages=input_data.pages.values[0:1]

In [7]:
s_c=[]
n_c=[]
d_c=[]
p_c=[]
u_c=[]
t_c=[]
for s in session:
    for number, date, page in numbers_dates:
        number=add_zeros(number)
        date=datetime.datetime.strptime(date, '%d/%m/%Y').strftime('%Y-%m-%d')
        for page_i in range(1,page+1):
                url_=f'https://debates.parlamento.pt/catalogo/r3/{legislature}/{cycle}/{s}/{number}/{date}/{page_i}'
                if validators.url(url_):
                    #print (url_)
                    u_c.append(url_)
                    p_c.append(page_i)
                    t_c.append(get_text_from_html(url_))
                    s_c.append(s)
                    n_c.append(number)
                    d_c.append(date)
df['page']=p_c
df['session']=s_c
df['number']=n_c
df['date']=d_c

df['text_1']=t_c
df['url']=u_c
df['legislature']=legislature
df['cycle']=cycle    

In [ ]:
#df.to_pickle('portuguese_transcripts.pkl') 
#df = pd.read_pickle('portuguese_transcripts.pkl')

In [8]:
df

,legislature,cycle,session,number,date,page,text_1,url
0,dar/01,14,01,001,2019-10-25,1,"<noscript>\n<p>Sábado, 26 de outubro de 2019 ...",https://debates.parlamento.pt/catalogo/r3/dar/...
1,dar/01,14,01,001,2019-10-25,2,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...
2,dar/01,14,01,001,2019-10-25,3,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>3 ...,https://debates.parlamento.pt/catalogo/r3/dar/...
3,dar/01,14,01,001,2019-10-25,4,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...
4,dar/01,14,01,001,2019-10-25,5,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>5 ...,https://debates.parlamento.pt/catalogo/r3/dar/...
5,dar/01,14,01,001,2019-10-25,6,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...
6,dar/01,14,01,001,2019-10-25,7,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>7 ...,https://debates.parlamento.pt/catalogo/r3/dar/...
7,dar/01,14,01,001,2019-10-25,8,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...
8,dar/01,14,01,001,2019-10-25,9,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>9 ...,https://debates.parlamento.pt/catalogo/r3/dar/...
9,dar/01,14,01,001,2019-10-25,10,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...


In [16]:
df['dialogs']=[check_dialog(t) for t in df.text_1]

In [72]:
df['speech']=[extract_dialog(t) for t in df.dialogs ]

In [82]:
df['party']=[extract_party_name(t) for t in df.speech  ]

In [83]:
df

,legislature,cycle,session,number,date,page,text_1,url,dialogs,speech,party
0,dar/01,14,01,001,2019-10-25,1,"<noscript>\n<p>Sábado, 26 de outubro de 2019 ...",https://debates.parlamento.pt/catalogo/r3/dar/...,None,None,None
1,dar/01,14,01,001,2019-10-25,2,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,</p><p></p><p></p><p>2 </p><p>Às 10 horas e 9 ...,[</p><p></p><p></p><p>2 </p><p>Às 10 horas e 9...,"[PS, Ana Catarina Mendonça Mendes ]"
2,dar/01,14,01,001,2019-10-25,3,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>3 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,None,None,None
3,dar/01,14,01,001,2019-10-25,4,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,None,None,None
4,dar/01,14,01,001,2019-10-25,5,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>5 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,None,None,None
5,dar/01,14,01,001,2019-10-25,6,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,"</p><p></p><p></p><p>6 </p><p>Essa chamada, al...","[</p><p></p><p></p><p>6 </p><p>Essa chamada, a...","[Duarte Pacheco, Secretário (Duarte Pacheco)]"
6,dar/01,14,01,001,2019-10-25,7,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>7 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,None,None,None
7,dar/01,14,01,001,2019-10-25,8,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,None,None,None
8,dar/01,14,01,001,2019-10-25,9,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>9 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,None,None,None
9,dar/01,14,01,001,2019-10-25,10,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,"</p><p></p><p></p><p>10 </p><p>Parabéns, Sr. P...","[</p><p></p><p></p><p>10 </p><p>Parabéns, Sr. ...","[[], AplausosPSD Presidente]"


In [65]:
pat="(?:</p><p>.*?\(.*?\): —.*?\: —)"
match=re.search(pat, df.text_1[0])
match

In [73]:
df[df.dialogs!='no_dialog'].dialogs[1]

'</p><p></p><p></p><p>2 </p><p>Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS): — Sr.as e Srs. Deputados, em nome da boa praxe parlamentar, </p><p>e sendo do grupo parlamentar maioritário nesta Assembleia da República, cabe-me iniciar os trabalhos desta </p><p>nova Legislatura. </p><p>Por isso mesmo, gostaria, em primeiro lugar, de saudar todas Sr.as Deputadas e todos os Srs. Deputados, </p><p>todos os eleitos, para que cumpramos aquele que é o respeito pela intuição democrática Assembleia da </p><p>República e para que cada um de nós exerça o seu mandato em representação daqueles que nos elegeram </p><p>para aqui estar. </p><p>Cumprindo também a praxe parlamentar, cabe-me, em nome do Grupo Parlamentar do Partido Socialista, </p><p>com muita honra também, designar para presidir aos nossos trabalhos, nesta primeira sessão, o Sr. Deputado </p><p>Eduardo Ferro Rodrigues, que é, de resto, o Presid

In [78]:
tt="</p><p></p><p></p><p>2 </p><p>Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS): — Sr.as e Srs. Deputados, em nome da boa praxe parlamentar, </p><p>e sendo do grupo parlamentar maioritário nesta Assembleia da República, cabe-me iniciar os trabalhos desta </p><p>nova Legislatura. </p><p>Por isso mesmo, gostaria, em primeiro lugar, de saudar todas Sr.as Deputadas e todos os Srs. Deputados, </p><p>todos os eleitos, para que cumpramos aquele que é o respeito pela intuição democrática Assembleia da </p><p>República e para que cada um de nós exerça o seu mandato em representação daqueles que nos elegeram </p><p>para aqui estar. </p><p>Cumprindo também a praxe parlamentar, cabe-me, em nome do Grupo Parlamentar do Partido Socialista, </p><p>com muita honra também, designar para presidir aos nossos trabalhos, nesta primeira sessão, o Sr. Deputado </p><p>Eduardo Ferro Rodrigues, que é, de resto, o Presidente da Assembleia da República cessante e a quem peço </p><p>que suba ao seu lugar. </p><p></p><p>Aplausos do PS, do BE, do PCP, do PEV e de Deputados do PSD. </p><p></p><p>Com o assentimento da Assembleia, assumiu a presidência o Deputado do PS Eduardo Ferro Rodrigues. </p><p></p><p>O Sr. Presidente: —Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS): — Sr.as e Srs. Deputados, em nome da boa praxe parlamentar, </p><p>e sendo do grupo parlamentar maioritário nesta Assembleia da República, cabe-me iniciar os trabalhos desta </p><p>nova Legislatura. </p><p>Por isso mesmo, gostaria, em primeiro lugar, de saudar todas Sr.as Deputadas e todos os Srs. Deputados, </p><p>todos os eleitos, para que cumpramos aquele que é o respeito pela intuição democrática Assembleia da </p><p>República e para que cada um de nós exerça o seu mandato em representação daqueles que nos elegeram </p><p>para aqui estar. </p><p>Cumprindo também a praxe parlamentar, cabe-me, em nome do Grupo Parlamentar do Partido Socialista, </p><p>com muita honra também, designar para presidir aos nossos trabalhos, nesta primeira sessão, o Sr. Deputado </p><p>Eduardo Ferro Rodrigues, que é, de resto, o Presidente da Assembleia da República cessante e a quem peço </p><p>que suba ao seu lugar. </p><p></p><p>Aplausos do PS, do BE, do PCP, do PEV e de Deputados do PSD. </p><p></p><p>Com o assentimento da Assembleia, assumiu a presidência o Deputado do PS Eduardo Ferro Rodrigues. </p><p></p><p>O Sr. Presidente: —'"

pat="(?:</p><p>.*?\(.*?\): —.*?\: —)"
match=re.findall(pat, tt)

In [18]:
ttt="</p><p></p><p></p><p>2 </p><p>Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS): — Sr.as e Srs. Deputados, em nome da boa praxe parlamentar, </p><p>e sendo do grupo parlamentar maioritário nesta Assembleia da República, cabe-me iniciar os trabalhos desta </p><p>nova Legislatura. </p><p>Por isso mesmo, gostaria, em primeiro lugar, de saudar todas Sr.as Deputadas e todos os Srs. Deputados, </p><p>todos os eleitos, para que cumpramos aquele que é o respeito pela intuição democrática Assembleia da </p><p>República e para que cada um de nós exerça o seu mandato em representação daqueles que nos elegeram </p><p>para aqui estar. </p><p>Cumprindo também a praxe parlamentar, cabe-me, em nome do Grupo Parlamentar do Partido Socialista, </p><p>com muita honra também, designar para presidir aos nossos trabalhos, nesta primeira sessão, o Sr. Deputado </p><p>Eduardo Ferro Rodrigues, que é, de resto, o Presidente da Assembleia da República cessante e a quem peço </p><p>que suba ao seu lugar. </p><p></p><p>Aplausos do PS, do BE, do PCP, do PEV e de Deputados do PSD. </p><p></p><p>Com o assentimento da Assembleia, assumiu a presidência o Deputado do PS Eduardo Ferro Rodrigues. </p><p></p><p>O Sr. Presidente: —eputados, </p><p>todos os eleitos, para que cumpramos aquele que é o respeito pela intuição democrática Assembleia da </p><p>República e para que cada um de nós exerça o seu mandato em representação daqueles que nos elegeram </p><p>para aqui estar. </p><p>Cumprindo também a praxe parlamentar, cabe-me, em nome do Grupo Parlamentar do Partido Socialista, </p><p>com muita honra também, designar para presidir aos nossos trabalhos, nesta primeira sessão, o Sr. Deputado </p><p>Eduardo Ferro Rodrigues, que é, de resto, o Presidente da Assembleia da República cessante e a quem peço </p><p>que suba ao seu lugar. </p><p></p><p>Aplausos do PS, do BE, do PCP, do PEV e de Deputados d<p>Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS): — Sr.as e Srs. Deputados, em nome da boa praxe parlamentar, </p><p>e sendo do grupo parlamentar maioritário nesta Assembleia da República, cabe-me iniciar os trabalhos desta </p><p>nova Legislatura. </p><p>Por isso mesmo, gostaria, em primeiro lugar, de saudar todas Sr.as Deputadas e todos os Srs. Deputados, </p><p>todos os eleitos, para que cumpramos aquele que é o respeito pela intuição democrática Assembleia da </p><p>República e para que cada um de nós exerça o seu mandato em representação daqueles que nos elegeram </p><p>para aqui estar. </p><p>Cumprindo também a praxe parlamentar, cabe-me, em nome do Grupo Parlamentar do Partido Socialista, </p><p>com muita honra também, designar para presidir aos nossos trabalhos, nesta primeira sessão, o Sr. Deputado </p><p>Eduardo Ferro Rodrigues, que é, de resto, o Presidente da Assembleia da República"
pat="(?:<p>.+\(.+\): —.+\: —)"
match=re.findall(pat, ttt)


In [91]:
def smallest_between_two(a, b, text):
    return re.findall(re.escape(a)+"(.*?)"+re.escape(b),text)

print(smallest_between_two('<p> ', '): —', ttt))
#print(smallest_between_two('<p>', ': —', '<p>this one: —]<p>[not this one: —'))
#print(smallest_between_two('paste ', '/', '@paste "game_01/01"'))

[]


In [19]:

m=re.findall('</p><p></p><p>(.+?): —', ttt)

In [20]:
m

['</p><p>2 </p><p>Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS)',
 'Aplausos do PS, do BE, do PCP, do PEV e de Deputados do PSD. </p><p></p><p>Com o assentimento da Assembleia, assumiu a presidência o Deputado do PS Eduardo Ferro Rodrigues. </p><p></p><p>O Sr. Presidente',
 'Aplausos do PS, do BE, do PCP, do PEV e de Deputados d<p>Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS)']

In [25]:
vec=[(m.start(0), m.end(0)) for m in re.finditer('</p><p></p><p>(.+?): —', ttt)]

In [27]:
vec

[(0, 174), (1084, 1304), (1915, 2130)]

In [36]:
m=extract_dialog( ttt)

In [28]:
[i for i in range(0,3)]

[0, 1, 2]

In [35]:
def extract_dialog(full_text):    
    dialogs=[(m.start(0), m.end(0)) for m in re.finditer('</p><p></p><p>(.+?): —', full_text)]
    out=[]
    if len(dialogs)==1:
        out.append(full_text[dialogs[0][0]:])
    if len(dialogs)>1:
        for i in range(0, len(dialogs)-1):
            out.append(full_text[dialogs[i][0]:dialogs[i+1][0]])
        out.append(full_text[dialogs[len(dialogs)-1][0]:])

    return out


In [45]:
m

['</p><p></p><p></p><p>2 </p><p>Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS): — Sr.as e Srs. Deputados, em nome da boa praxe parlamentar, </p><p>e sendo do grupo parlamentar maioritário nesta Assembleia da República, cabe-me iniciar os trabalhos desta </p><p>nova Legislatura. </p><p>Por isso mesmo, gostaria, em primeiro lugar, de saudar todas Sr.as Deputadas e todos os Srs. Deputados, </p><p>todos os eleitos, para que cumpramos aquele que é o respeito pela intuição democrática Assembleia da </p><p>República e para que cada um de nós exerça o seu mandato em representação daqueles que nos elegeram </p><p>para aqui estar. </p><p>Cumprindo também a praxe parlamentar, cabe-me, em nome do Grupo Parlamentar do Partido Socialista, </p><p>com muita honra também, designar para presidir aos nossos trabalhos, nesta primeira sessão, o Sr. Deputado </p><p>Eduardo Ferro Rodrigues, que é, de resto, o Presi

In [65]:

def extract_party_name(dialogs):
    for i in dialogs:
        positions=[(m.start(0), m.end(0)) for m in re.finditer('(.+?): —', i)]
        party=re.findall('\((.+?)\)',i)
        name_aux=i[positions[0][1]-50:positions[0][1]]
        name=re.sub(r'\b\w{1,2}\b', '', name_aux).replace('.','').replace ('()','').replace('  ','').replace('<','').replace('>','').replace('/','').replace(': —','')
        return [party[0], name]
    

In [66]:
extract_party_name(m)

['PS', 'Ana Catarina Mendonça Mendes ']

In [55]:
i

'</p><p></p><p>Aplausos do PS, do BE, do PCP, do PEV e de Deputados d<p>Às 10 horas e 9 minutos, tomou a palavra a Deputada do PS Ana Catarina Mendonça Mendes.</p><p></p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS): — Sr.as e Srs. Deputados, em nome da boa praxe parlamentar, </p><p>e sendo do grupo parlamentar maioritário nesta Assembleia da República, cabe-me iniciar os trabalhos desta </p><p>nova Legislatura. </p><p>Por isso mesmo, gostaria, em primeiro lugar, de saudar todas Sr.as Deputadas e todos os Srs. Deputados, </p><p>todos os eleitos, para que cumpramos aquele que é o respeito pela intuição democrática Assembleia da </p><p>República e para que cada um de nós exerça o seu mandato em representação daqueles que nos elegeram </p><p>para aqui estar. </p><p>Cumprindo também a praxe parlamentar, cabe-me, em nome do Grupo Parlamentar do Partido Socialista, </p><p>com muita honra também, designar para presidir aos nossos trabalhos, nesta primeira sessão, o Sr. Deputado </p><p>Eduardo F

In [43]:
len('A Sr.ª Ana Catarina Mendonça Mendes (PS)')

40

In [49]:

[(m.start(0), m.end(0)) for m in re.finditer('(.+?): —', m[0])]

[(0, 174)]

In [50]:
m[0][174-50:174]

'</p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS): —'

In [102]:
c=m[0][m[0].index('</p><p></p><p>')+len('</p><p></p><p>'):]

In [106]:
party=re.findall('\((.+?)\)',c)

['PS']

In [109]:
re.sub(r'\b\w{1,2}\b', '', c).replace('.','').replace ('()','').replace('  ','')

'Ana Catarina Mendonça Mendes '